**как должно работать:**

- формируем пул буферов: n штук заполненых последовательно из экземпляра Random
- поддерживаем кол-во буферов:
   - по мере расходования, когда вынули буфер из пула, даём команду на формирование ещё одного буфера
   - пулинг
- нужен класс (лучше с генериком для поддержки double, integer и т.д.), экземпляр которого:
   1) хранит буфер случайных чисел
   2) может отдавать следующее случайное число, вне зависимости откуда пришёл вызов, либо сообщать что буфер закончился
   3) возвращает буфер в пул, либо сам возвращается в пул

- нужен статический диспетчер, который:
   - держит коллекцию вышеобозначенных объектов
   - имеет статический метод для выдачи случайных чисел:
      1) при вызове выбираем один из объектов
      2) вызваем у выбранного объекта метод получения случайного числа
      3) если овет что у объекта буфер закончился, то:
         - запускаем отдельную таску по восстановлению объекрта
         - выполняем п. 1
   - например, распорядитель может у себя держать очередь из объектов; при вызове метода получения следующего случайного числа, распорядитель:
      1. вынимает объект из очереди, или создаёт новый
      2. вызывает у полученного объекта метод получения следующего случайного числа
      3.
         - если получено число - возвращаем объект обратно в очередь
         - если буфер закончился - добавляем объект в отдельную очередь на восстановление

In [ ]:
record struct NextResult<T> (bool Success, T Value);

static class NextResult
{
    public static NextResult<T> Success<T>(T value) => new NextResult<T>(true, value);
    public static NextResult<T> Fail<T>() => new NextResult<T>(false, default);
}

In [ ]:
class BufferKeeper<T>
{
    private T[] _buffer;
    private int _index = -1;

    public BufferKeeper(int bufferSize)
    {
        _buffer = new T[bufferSize];
    }

    public NextResult<T> Next()
    {
        lock (this)
        {
            if (++_index >= _buffer.Length)
                return NextResult.Fail<T>();
            
            return NextResult.Success(_buffer[_index]);
        }
    }

    public bool HasValues()
    {
        lock (this)
        {
            return _index < _buffer.Length;
        }
    }

    public void Restore(Func<T> next)
    {
        lock (this)
        {
            if (_index == 0)
                return;
            
            for (var i = 0; i < _buffer.Length; ++i)
                _buffer[i] = next();
            
            _index = 0;
        }
    }
}

In [ ]:
using System.Threading.Tasks;

class SequentionalScheduler
{
    private Task _prevTask = Task.CompletedTask;

    public Task Run(Action job)
    {
        lock (this)
        {
            return _prevTask = _prevTask.ContinueWith(t => job());
        }
    }

    public Task Run(Action<Task, object> job, object o)
    {
        lock (this)
        {
            return _prevTask = _prevTask.ContinueWith(job, o);
        }
    }

    public Task Run(Func<Task> job)
    {
        lock (this)
        {
            return _prevTask = _prevTask.ContinueWith(async t => await job());
        }
    }
}

In [ ]:
using System.Collections.Concurrent;

abstract class ARandomValuesDispatcher<T>
{
    private readonly ConcurrentQueue<BufferKeeper<T>> _buffersQueue = new ConcurrentQueue<BufferKeeper<T>>();
    private readonly SequentionalScheduler _scheduler = new SequentionalScheduler();
    private readonly int _bufferSize;

    public TimeSpan RetryPause { get; set; } = TimeSpan.FromMilliseconds(5);

    public ARandomValuesDispatcher(uint queueSize, int bufferSize)
    {
        _bufferSize = bufferSize;
        for (var i = 0; i < queueSize; ++i)
        {
            RestoreKeeper(new BufferKeeper<T>(bufferSize));
        }
    }

    public T Next()
    {
        while (true)
        {
            BufferKeeper<T> keeper;
            while (!_buffersQueue.TryDequeue(out keeper));

            var result = keeper.Next();
            if (result.Success)
                try
                {
                    return result.Value;
                }
                finally
                {
                    Task.Factory.StartNew(() => {
                        if (keeper.HasValues())
                            _buffersQueue.Enqueue(keeper);
                        else
                            ScheduleKeeperRestoration(keeper);
                    });
                }
            else
                ScheduleKeeperRestoration(keeper);
        }
    }

    public async ValueTask<T> SmartNext()
    {
        while (true)
        {
            BufferKeeper<T> keeper;
            var i = 0;
            while (!_buffersQueue.TryDequeue(out keeper))
            {
                if (++i == 10)
                    ScheduleKeeperRestoration(new BufferKeeper<T>(_bufferSize));
                await Task.Delay(RetryPause);
            }

            var result = keeper.Next();
            if (result.Success)
                try
                {
                    return result.Value;
                }
                finally
                {
                    Task.Factory.StartNew(() => {
                        if (keeper.HasValues())
                            _buffersQueue.Enqueue(keeper);
                        else
                            ScheduleKeeperRestoration(keeper);
                    });
                }
            else
                ScheduleKeeperRestoration(keeper);
        }
    }

    public async IAsyncEnumerable<T> Enumerate()
    {
        while (true)
        {
            BufferKeeper<T> keeper;
            var i = 0;
            while (!_buffersQueue.TryDequeue(out keeper))
            {
                if (++i == 10)
                    ScheduleKeeperRestoration(new BufferKeeper<T>(_bufferSize));
                await Task.Delay(RetryPause);
            }
            
            var result = keeper.Next();
            while (result.Success)
            {
                yield return result.Value;
                result = keeper.Next();
            }
            
            ScheduleKeeperRestoration(keeper);
        }
    }

    abstract protected T GenerateRandomValue();

    private void RestoreKeeper(BufferKeeper<T> keeper)
    {
        keeper.Restore(GenerateRandomValue);
        _buffersQueue.Enqueue(keeper);
    }
    private void RestoreKeeper(Task _, object keeper) => RestoreKeeper((BufferKeeper<T>)keeper);
    private void ScheduleKeeperRestoration(BufferKeeper<T> keeper) => _scheduler.Run(RestoreKeeper, keeper);

    //public static Dispatcher Default { get; } = new Dispatcher(11);
}

/* todo: overloads for Next:
    (T, T) Next2()
    (T, T, T) Next3()
    ...
    T[] NextArray(int arraySize) via ArrayPool<T>
*/
// todo: init ArrayPool<T>(int expectetArraySize)

In [ ]:
class RandomDoublesDispatcher : ARandomValuesDispatcher<double>
{
    public RandomDoublesDispatcher(uint queueSize, int bufferSize)
        : base(queueSize, bufferSize)
    {}

    private Random _random = new Random();
    override protected double GenerateRandomValue() => _random.NextDouble();
}

In [ ]:
var random = new RandomDoublesDispatcher(32, 1024);

Enumerable.Range(1, 19).AsParallel().Select(i => new { i, f = random.SmartNext().Result, })

index,i,f
0,7,0.6745169304554197
1,1,0.421430138455283
2,2,0.3071251046351373
3,4,0.07915111199007419
4,6,0.02184618358559498
5,8,0.6902024097420224
6,17,0.9999870312024891
7,18,0.45169375137867607
8,9,0.4507309906767031
9,10,0.5813048468323427
